question answering example doing the following:

1. load the deepset model stored on huggingface via eland into elasticsearch
2. setup a pipeline + ingest data (wikipedia) to enrich
3. ask questions on dataset
